# Prepare for each projects

In [ ]:
!pip install fabric

In [ ]:
!pip install python-dotenv

In [ ]:
from fabric import Connection, Config

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [ ]:
pwd = os.getenv('PI_PSWD')
pi1 = os.getenv('PI_IP1')

In [ ]:
config = Config({'user': 'pi', 'connect_kwargs': {'password': pwd}})

In [ ]:
cxn = Connection(pi1, config = config)

In [ ]:
cxn.run('whoami')

In [ ]:
cxn.sudo('pip3 install --upgrade pip')

In [ ]:
# cxn.sudo('pip install virtualenv')

# Redis Server (Docker)

In [ ]:
import requests
from redis import Redis
from rq import Queue
from rq_scheduler import Scheduler

from datetime import datetime
import pytz

In [ ]:
# cxn.run('redis-server --version')

In [ ]:
host = 'redis_local' #os.getenv('REDIS_HOST')
port = os.getenv('REDIS_PORT')

In [ ]:
r = Redis(host, port)

In [ ]:
r

In [ ]:
r.set('example', '1928')

In [ ]:
r.get('example')

In [ ]:
scheduler = Scheduler(connection = r)

In [ ]:
scheduler.count()

# Redis Raspberry pi

In [ ]:
host = os.getenv('REDIS_HOST')
port = os.getenv('REDIS_PORT')
redis_password = os.getenv('REDIS_PASSWORD_PI')

In [ ]:
# cxn.sudo('pip3 install redis')

In [ ]:
# cxn.sudo('apt-get install redis -y')

In [ ]:
# cxn.sudo('kill -9 32008  ') # Not working
#cxn.sudo('service redis-server stop')

In [ ]:
# cxn.sudo('ps aux | grep redis')

In [ ]:
# cxn.sudo(f'service redis-server restart ./redis.conf') #service redis-server restart
# cxn.sudo('service redis-server restart --protected-mode no')
cxn.sudo('redis-server --protected-mode no')

In [ ]:
r = Redis(host, port)

In [ ]:
r.set('foo', 'bar1', px = 100_000)

In [ ]:
r.get('foo')

In [ ]:
r.ttl('foo')

In [ ]:
r.sadd('pythonlist', '1', '2', '3', '4')

In [ ]:
r.sadd('powershelllist', '2', '4')

In [ ]:
r.sinter('pythonlist', 'powershelllist')

In [ ]:
r.sunion('pythonlist', 'powershelllist')

In [ ]:
r.scard('pythonlist')

In [ ]:
r.expire('pythonlist', 1)

In [ ]:
r.expire('powershelllist',1)

# Saving pandas to Redis

In [ ]:
import pyarrow as pa
import pandas as pd

In [ ]:
def storeInRedis(alias, df):
    df_compressed = pa.serialize(df).to_buffer().to_pybytes()
    res = r.set(alias,df_compressed)
    if res == True:
        print(f'{alias} cached')

def loadFromRedis(alias):
    data = r.get(alias)
    try:
        return pa.deserialize(data)
    except:
        print("No data")

In [ ]:
dd = {'ID': ['H576','H577','H578','H600', 'H700'],
      'CD': ['AAAAAAA', 'BBBBB', 'CCCCCC','DDDDDD', 'EEEEEEE']}
df = pd.DataFrame(dd)

In [ ]:
storeInRedis('pandas', df)

In [ ]:
loadFromRedis('pandas')

In [ ]:
r.expire('pandas', 10)